<a href="https://colab.research.google.com/github/Noble-mungu/RTFS/blob/master/Real_time_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install faker


In [ ]:
from faker import Faker
import pandas as pd

fake = Faker()

def generate_accounts(num=600):
    accounts = []
    for _ in range(num):
        accounts.append({
            "cc_num": fake.credit_card_number(),
            "first": fake.first_name(),
            "last": fake.last_name(),
            "gender": fake.random_element(elements=('Male', 'Female')),
            "street": fake.street_address(),
            "city": fake.city(),
            "state": fake.state(),
            "zip": fake.zipcode(),
            "dob": fake.date_of_birth().isoformat(),
            "job": fake.job(),
            "lat": fake.latitude(),
            "long": fake.longitude(),
        })
    return pd.DataFrame(accounts)

if __name__ == "__main__":
    accounts_df = generate_accounts()
    accounts_df.to_csv('accounts.csv', index=False)


In [ ]:
from faker import Faker
import pandas as pd

fake = Faker()

def generate_sanctioned_accounts(num=150):
    sanctioned = []
    for _ in range(num):
        sanctioned.append({
            "cc_num": fake.credit_card_number(),
            "first": fake.first_name(),
            "last": fake.last_name(),
            "reason": fake.sentence(),
        })
    return pd.DataFrame(sanctioned)

if __name__ == "__main__":
    sanctioned_df = generate_sanctioned_accounts()
    sanctioned_df.to_csv('sanctioned.csv', index=False)


In [ ]:
from faker import Faker
import pandas as pd
import random
from datetime import datetime, timedelta

fake = Faker()

def generate_transactions(num=18000):
    transactions = []
    for _ in range(num):
        transaction_date = fake.date_time_this_year()
        is_fraud = random.choice([True, False])
        transactions.append({`
            "cc_num": fake.credit_card_number(),
            "trans_num": fake.uuid4(),
            "trans_date": transaction_date.isoformat(),
            "amt": round(random.uniform(1.0, 1000.0), 2),
            "merchant": fake.company(),
            "is_fraud": is_fraud,
        })
    return pd.DataFrame(transactions)

if __name__ == "__main__":
    transactions_df = generate_transactions()
    transactions_df.to_csv('transactions.csv', index=False)


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving mongo-spark-connector_2.12-3.0.1-assembly.jar to mongo-spark-connector_2.12-3.0.1-assembly.jar


In [ ]:
pip install "pymongo[srv]"

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_date, datediff
from pyspark.sql.types import DateType

database_name = "Real-time-fraud"
atlas_connection_string = f"mongodb+srv://mungunoble:tj5>6QWB$3Z#rFh@cluster1.y2irkap.mongodb.net/{database_name}?retryWrites=true&w=majority&appName=Cluster1"

# Initialize Spark Session
# spark = SparkSession.builder \
#     .appName("Real-time-fraud") \
#     .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
#     .getOrCreate()

mongo_spark_connector_path = "/content/mongo-spark-connector_2.12-3.0.1-assembly.jar"

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Real-time-fraud") \
    .config("spark.mongodb.input.uri", f"{atlas_connection_string}&collection=accounts") \
    .config("spark.mongodb.output.uri", f"{atlas_connection_string}&collection=accounts") \
    .config("spark.mongodb.input.uri", f"{atlas_connection_string}&collection=sanctioned") \
    .config("spark.mongodb.output.uri", f"{atlas_connection_string}&collection=sanctioned") \
    .config("spark.mongodb.input.uri", f"{atlas_connection_string}&collection=transactions") \
    .config("spark.mongodb.output.uri", f"{atlas_connection_string}&collection=transactions") \
    .config("spark.jars", mongo_spark_connector_path) \
    .getOrCreate()
# Load CSV files
accounts_df = spark.read.csv("accounts.csv", header=True, inferSchema=True)
sanctioned_df = spark.read.csv("sanctioned.csv", header=True, inferSchema=True)
transactions_df = spark.read.csv("transactions.csv", header=True, inferSchema=True)

# Ensure 'dob' column is of DateType in accounts_df
accounts_df = accounts_df.withColumn("dob", col("dob").cast(DateType()))

# Join transactions_df with accounts_df to include dob
transactions_df = transactions_df.join(accounts_df.select("cc_num", "dob"), on="cc_num", how="left")

# Calculate age using the difference between current date and dob
transactions_df = transactions_df.withColumn("age", (datediff(current_date(), col("dob")) / 365).cast("int")) \
                                   .withColumn("distance", lit(0))  # Placeholder for distance

# Write data to MongoDB with explicit collection options
accounts_df.write.format("mongo").mode("append") \
    .option("uri", atlas_connection_string) \
    .option("database", database_name) \
    .option("collection", "accounts") \
    .save()

sanctioned_df.write.format("mongo").mode("append") \
    .option("uri", atlas_connection_string) \
    .option("database", database_name) \
    .option("collection", "sanctioned") \
    .save()

transactions_df.write.format("mongo").mode("append") \
    .option("uri", atlas_connection_string) \
    .option("database", database_name) \
    .option("collection", "transactions") \
    .save()

# Stop Spark session
spark.stop()


Py4JJavaError: An error occurred while calling o814.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 101.0 failed 1 times, most recent failure: Lost task 0.0 in stage 101.0 (TID 101) (d275e055a0a6 executor driver): com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting for a server that matches com.mongodb.client.internal.MongoClientDelegate$1@66a73f5b. Client view of cluster state is {type=REPLICA_SET, servers=[{address=ac-8nbtlmy-shard-00-01.y2irkap.mongodb.net:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketWriteException: Exception sending message}, caused by {javax.net.ssl.SSLException: Received fatal alert: internal_error}}, {address=ac-8nbtlmy-shard-00-00.y2irkap.mongodb.net:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketWriteException: Exception sending message}, caused by {javax.net.ssl.SSLException: Received fatal alert: internal_error}}, {address=ac-8nbtlmy-shard-0

In [ ]:
!pip install pymongo pyspark pymongo[srv]
